In [10]:
import tensorflow as tf
import cv2
import os

# Load image and annotation
image_path = '/Users/hassanazhar/Python/damage_detect/Damage_Images/image01.jpg'
annotation_path = '/Users/hassanazhar/Python/damage_detect/label_images/image01.png'

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

# Example function to parse annotations
def parse_annotations(annotation_path):
    # Implement this function to parse your annotations
    # and return bounding boxes and labels
    annotations = pd.read_csv('annotations.csv')
    
    # Dictionary to store the annotations for each image
    annotation_dict = {}
    
    for index, row in annotations.iterrows():
        filename = row['filename']
        bbox = [row['xmin'], row['ymin'], row['xmax'], row['ymax']]
        label = row['label']
        
        if filename not in annotation_dict:
            annotation_dict[filename] = {'boxes': [], 'labels': []}
        
        annotation_dict[filename]['boxes'].append(bbox)
        annotation_dict[filename]['labels'].append(label)
    return annotation_dict[filename]['boxes'], annotation_dict[filename]['labels']

boxes, labels = parse_annotations(annotation_path)

In [12]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [17]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    '/Users/hassanazhar/Python/damage_detect/Damage_Images',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = datagen.flow_from_directory(
    '/Users/hassanazhar/Python/damage_detect/label_images',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size,
    epochs=10
)

Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.


ValueError: Must provide at least one structure

In [16]:
for layer in base_model.layers[:100]:
    layer.trainable = True

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [3]:
import cv2

def load_and_preprocess_image(image_path):
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError(f"Failed to load image at {image_path}.")
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    return image

image_path = '/Users/hassanazhar/Python/damage_detect/Damage_Images'
image = load_and_preprocess_image(image_path)

ValueError: Failed to load image at /Users/hassanazhar/Python/damage_detect/Damage_Images.

In [5]:
import pandas as pd

# Example data
data = [
    {'filename': 'image01.png', 'xmin': 50, 'ymin': 30, 'xmax': 200, 'ymax': 180, 'label': 'damage'},
    {'filename': 'image02.png', 'xmin': 100, 'ymin': 100, 'xmax': 300, 'ymax': 300, 'label': 'damage'},
    {'filename': 'image03.png', 'xmin': 30, 'ymin': 50, 'xmax': 100, 'ymax': 200, 'label': 'damage'}
    # Add more annotations as needed
]

# Convert to DataFrame
df = pd.DataFrame(data)

# Save to CSV
df.to_csv('annotations.csv', index=False)